In [118]:
from acquire import get_telco_data
from prepare import train_validate_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import explore
from scipy import stats

In [139]:
df = get_telco_data()

Using cached data


In [140]:
df.shape

(7043, 24)

# Prepare Exploration (Pre-split)

In [141]:
# Initially drop redundant columns
df = df.drop(columns = ['payment_type_id','internet_service_type_id','contract_type_id'])

In [142]:
# From prior exploration of dataset a small number of the total_charges are just whitespace - these are all new customers who haven't been with the company for >1 month.
# Given that it is a very small proportion of the total dataset these rows will be deleted for ease of computation later on
(df == " ").sum()

customer_id               0
gender                    0
senior_citizen            0
partner                   0
dependents                0
tenure                    0
phone_service             0
multiple_lines            0
online_security           0
online_backup             0
device_protection         0
tech_support              0
streaming_tv              0
streaming_movies          0
paperless_billing         0
monthly_charges           0
total_charges            11
churn                     0
contract_type             0
internet_service_type     0
payment_type              0
dtype: int64

In [143]:
df = df.drop(df[df.total_charges == " "].index)

In [144]:
# Confirm no more blanks
(df == " ").sum().sum()

0

In [145]:
# Confirm no nulls
(df.isna().sum()).sum()

0

In [146]:
# Convert total_charges to float for later analysis
df.total_charges = df.total_charges.astype('float64')

In [147]:
# Confirm dtypes
df.dtypes

customer_id               object
gender                    object
senior_citizen             int64
partner                   object
dependents                object
tenure                     int64
phone_service             object
multiple_lines            object
online_security           object
online_backup             object
device_protection         object
tech_support              object
streaming_tv              object
streaming_movies          object
paperless_billing         object
monthly_charges          float64
total_charges            float64
churn                     object
contract_type             object
internet_service_type     object
payment_type              object
dtype: object

In [154]:
df.internet_service_type.value_counts()

Fiber optic    3096
DSL            2416
None           1520
Name: internet_service_type, dtype: int64

In [156]:
df.payment_type.value_counts()

Electronic check             2365
Mailed check                 1604
Bank transfer (automatic)    1542
Credit card (automatic)      1521
Name: payment_type, dtype: int64

In [148]:
df.phone_service = df.phone_service.map({'Yes':1,'No':0})
df.churn = df.churn.map({'Yes':1,'No':0})
df.partner = df.partner.map({'Yes':1,'No':0})
df.dependents = df.dependents.map({'Yes':1,'No':0})
df.paperless_billing = df.paperless_billing.map({'Yes':1,'No':0})
df.contract_type = df.contract_type.map({'Two year':0,'One year':0})
df.payment_type = df.payment_type.map({

In [149]:
# # Determine the categorical variables - here defined as object data type (non-numeric) and with fewer than 5 values
# catcol = df.columns[(df.nunique()<5)&(df.dtypes == 'object')]
# # Encode categoricals
# dummy_df = pd.get_dummies(df[catcol], dummy_na=False, drop_first=True)
# # Concatenate dummy df to original df
# df = pd.concat([df,dummy_df],axis=1)
# # Remove the original categorical columns after encoding
# df = df.drop(columns=catcol)

# Exploration (Post-split)

In [150]:
train, validate, test = train_validate_test_split(df, target = 'churn')

In [151]:
train.shape, validate.shape, test.shape

((3937, 21), (1688, 21), (1407, 21))

In [152]:
train.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type'],
      dtype='object')

In [153]:
# Baseline churn rate
(train.churn == 1).mean()

0.2656845313690627

## Do any groups churn at rates significantly above the overall mean?

### Groups defined as:
    - senior citizen
    - male vs female
    - partner vs no partner
    - dependent vs no dependent
    - Auto vs manual payers
    - Paperless billing vs not
    - Month to month vs long term
    

In [137]:
train.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type'],
      dtype='object')

In [136]:
groups = ['gender', 'senior_citizen', 'partner', 'dependents', 'paperless_billing'
p, t = stats.ttest_1samp(train.churn[train.senior_citizen == 1], train.churn.mean())
p/2, t

(4.100283109621952, 1.3314799076710632e-15)

## Categorical variables

### Takeaways:
- Evenly distributed by gender and partner
- Two broad categories lead to subcategories that would be worthwhile to look at individually:
    - Phone service or not: 
        - Multiple lines
    - Internet Service or not:
        - online security
        - online backup
        - tech support
        - device protection
        - streaming tv
        - streaming movies
        - service type (DSL vs Fiber)
     

In [19]:
target = 'churn'

In [12]:
catcol

Index(['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn',
       'contract_type', 'internet_service_type', 'payment_type'],
      dtype='object')

In [30]:
train['partner']

5310    Yes
3790    Yes
4398    Yes
2635     No
2986    Yes
       ... 
6230    Yes
356      No
2128    Yes
3586     No
6817    Yes
Name: partner, Length: 3943, dtype: object